In [1]:
import pandas as pd
import re
import seaborn as sns
import requests
import json
from pandas import json_normalize
import re

In [2]:
df = pd.read_csv('archive/ghg-emissions.csv')

In [3]:
df.head()

,Country/Region,unit,1990,1991,1992,1993,1994,1995,1996,1997,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,China,MtCO2e,2873.71,3023.38,3154.54,3377.79,3528.81,3918.30,3954.75,3963.61,...,9046.19,9872.37,10364.83,10690.04,11144.76,11155.76,11149.68,11207.66,11408.26,11705.81
1,United States,MtCO2e,5543.47,5501.92,5583.80,5691.26,5782.20,5843.90,6010.62,6258.47,...,5793.66,6041.59,5796.78,5550.87,5690.79,5711.15,5586.69,5676.92,5613.64,5794.35
2,India,MtCO2e,1009.44,1064.49,1096.71,1128.44,1174.78,1240.44,1289.63,1348.09,...,2467.78,2576.93,2610.32,2758.53,2816.49,2988.34,3002.17,3073.24,3202.82,3346.63
3,Russia,MtCO2e,2885.12,2788.70,2551.94,2294.62,2011.47,1930.88,1889.18,1755.59,...,1575.54,1690.93,1864.86,1886.41,1844.48,1841.26,1791.21,1918.26,1939.26,1992.08
4,Indonesia,MtCO2e,1257.10,1274.43,1295.01,1311.27,1327.87,1360.88,1179.00,2142.82,...,1483.61,1125.25,1675.72,1695.27,1636.17,2016.28,2070.24,1453.23,1466.05,1703.86


In [4]:
# todos los nulls que nos encontramos estan en las dos ultimas columnas del pais/region por lo tanto podemos obviarlas
# todos los años tienen po ello los mismos nulls

In [5]:
df.tail()

,Country/Region,unit,1990,1991,1992,1993,1994,1995,1996,1997,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
190,Nauru,MtCO2e,0.13,0.13,0.13,0.12,0.12,0.11,0.11,0.11,...,0.05,0.05,0.05,0.05,0.05,0.06,0.06,0.07,0.07,0.08
191,Tuvalu,MtCO2e,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,...,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.03,0.03
192,Niue,MtCO2e,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.00,0.00,0.00,0.01,0.01,0.01,0.01,0.01
193,Data source,CAIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,Data for,Countries/Regions: World. Sectors/Subsectors: ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# vemos que aqui las dos ultimas columnas donde estan los nan no son realmente paises, por ello podemos quitarlas

In [7]:
#me voy a cargar las dos ultimas filas, a la mierda

In [8]:
df.drop([193,194], axis=0, inplace=True)

In [9]:
df.tail()

,Country/Region,unit,1990,1991,1992,1993,1994,1995,1996,1997,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
188,Kiribati,MtCO2e,0.04,0.04,0.04,0.05,0.04,0.05,0.05,0.05,...,0.08,0.08,0.08,0.08,0.09,0.09,0.09,0.10,0.10,0.11
189,Cook Islands,MtCO2e,0.03,0.03,0.03,0.03,0.03,0.04,0.04,0.05,...,0.07,0.07,0.08,0.08,0.08,0.08,0.08,0.09,0.09,0.10
190,Nauru,MtCO2e,0.13,0.13,0.13,0.12,0.12,0.11,0.11,0.11,...,0.05,0.05,0.05,0.05,0.05,0.06,0.06,0.07,0.07,0.08
191,Tuvalu,MtCO2e,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,...,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.03,0.03
192,Niue,MtCO2e,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.01,0.00,0.00,0.00,0.01,0.01,0.01,0.01,0.01


#### VAMOS A EMPEZAR A LLAMR A LA API YA!!!

In [10]:
url = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country')

In [11]:
url

<Response [200]>

In [12]:
url.json()

[{'name': 'Afghanistan', 'iso3': 'AFG', 'iso2': 'AF', 'id': 466},
 {'name': 'Albania', 'iso3': 'ALB', 'iso2': 'AL', 'id': 582},
 {'name': 'Algeria', 'iso3': 'DZA', 'iso2': 'DZ', 'id': 520},
 {'name': 'American Samoa (U.S.)', 'iso3': 'ASM', 'id': 677},
 {'name': 'Andorra', 'iso3': 'AND', 'id': 675},
 {'name': 'Angola', 'iso3': 'AGO', 'iso2': 'AO', 'id': 544},
 {'name': 'Anguilla (U.K.)', 'iso3': 'AIA', 'id': 716},
 {'name': 'Antigua and Barbuda', 'iso3': 'ATG', 'iso2': 'AG', 'id': 583},
 {'name': 'Argentina', 'iso3': 'ARG', 'iso2': 'AR', 'id': 563},
 {'name': 'Armenia', 'iso3': 'ARM', 'iso2': 'AM', 'id': 584},
 {'name': 'Aruba (Neth.)', 'iso3': 'ABW', 'id': 674},
 {'name': 'Australia', 'iso3': 'AUS', 'iso2': 'AU', 'id': 585},
 {'name': 'Austria', 'iso3': 'AUT', 'iso2': 'AT', 'id': 586},
 {'name': 'Azerbaijan', 'iso3': 'AZE', 'iso2': 'AZ', 'id': 491},
 {'name': 'Bahamas, The', 'iso3': 'BHS', 'iso2': 'BS', 'id': 587},
 {'name': 'Bahrain', 'iso3': 'BHR', 'iso2': 'BH', 'id': 588},
 {'name':

## ¿Qué daotos vamos a querer buscar para después comprarlos con los datos de nuestro dataset?
#### Temperaturas y precipitaciones de entre los años 1980 y 1999 de USA, Alemania, China, India y Rusia (potencias mundiales y/o paises altamente poblados)
#### Depués se buscarán los datos predictivos de años venideros de la temperatura de estos mismos países

In [13]:
usa_temp_1980_1999 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/tas/1980/1999/USA')

In [14]:
# en este caso me estan dando la media de temperatura anual de entre el 1980 y el 1999 de USA

In [15]:
df_oldtemp_usa = json_normalize(usa_temp_1980_1999.json())

In [16]:
df_oldtemp_usa

,gcm,variable,fromYear,toYear,annualData
0,bccr_bcm2_0,tas,1980,1999,[4.450069094534607]
1,cccma_cgcm3_1,tas,1980,1999,[5.971076870201408]
2,cnrm_cm3,tas,1980,1999,[6.263146987877741]
3,csiro_mk3_5,tas,1980,1999,[10.198723619419134]
4,gfdl_cm2_0,tas,1980,1999,[3.8936500454188594]
5,gfdl_cm2_1,tas,1980,1999,[5.286517668840554]
6,ingv_echam4,tas,1980,1999,[8.587654703573495]
7,inmcm3_0,tas,1980,1999,[6.68783409518294]
8,ipsl_cm4,tas,1980,1999,[7.3364575925901585]
9,miroc3_2_medres,tas,1980,1999,[7.460112764353939]


In [17]:
# lo que me estan dando son las diferentes mediciones de los centros donde han recogido la informacion
# lo que se puede hacer entonces es una media de eso mismo... o mediana

In [18]:
# voy a hacer una nueva columna que sea from year + to year y me de como resultado 1980-1999

In [19]:
df_oldtemp_usa['Interval_Years'] = '1980 1999'

In [20]:
df_oldtemp_usa['Annual_Data'] = [i[0] for i in df_oldtemp_usa['annualData']]

In [21]:
df_oldtemp_usa.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)

In [22]:
#ahora hacemos un gropuby por los años, con la media de Annual_Data, ya que las mediciones sabemos que son de 
#diferentes estaciones tan solo

In [23]:
grouped_oldtempusa = df_oldtemp_usa.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})


In [24]:
grouped_oldtempusa

,Interval_Years,Annual_Data
0,1980 1999,6.344341


In [25]:
#ahora es hacer los mismos pasos con todo el resto de los graficos
#llamar a la api con lo que queremos, y reducir al intervalo y  la media de la data

In [26]:
#precipitaciones de entre los años 1980 y 1999 en USA

In [27]:
usa_prec_1980_1999 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/pr/1980/1999/USA')

In [28]:
df_oldprec_usa = json_normalize(usa_prec_1980_1999.json())

In [29]:
df_oldprec_usa

,gcm,variable,fromYear,toYear,annualData
0,bccr_bcm2_0,pr,1980,1999,[785.2701232986692]
1,cccma_cgcm3_1,pr,1980,1999,[777.7889471267924]
2,cnrm_cm3,pr,1980,1999,[947.5967851291585]
3,csiro_mk3_5,pr,1980,1999,[798.381580457504]
4,gfdl_cm2_0,pr,1980,1999,[890.4286021472773]
5,gfdl_cm2_1,pr,1980,1999,[827.3124179982142]
6,ingv_echam4,pr,1980,1999,[851.9327652092476]
7,inmcm3_0,pr,1980,1999,[843.0909232427455]
8,ipsl_cm4,pr,1980,1999,[900.6203651333254]
9,miroc3_2_medres,pr,1980,1999,[819.1997264378206]


In [30]:
df_oldprec_usa['Interval_Years'] = '1980 1999'

In [31]:
df_oldprec_usa['Annual_Data'] = [i[0] for i in df_oldprec_usa['annualData']]

In [32]:
df_oldprec_usa.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)

In [33]:
grouped_oldprecusa = df_oldprec_usa.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})

In [34]:
grouped_oldprecusa

,Interval_Years,Annual_Data
0,1980 1999,841.912621


In [35]:
# temperatura en Alemania entre los años 1980 y 1999

In [36]:
deu_temp_1980_1999 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/tas/1980/1999/DEU')
df_oldtemp_deu = json_normalize(deu_temp_1980_1999.json())
df_oldtemp_deu['Interval_Years'] = '1980 1999'
df_oldtemp_deu['Annual_Data'] = [i[0] for i in df_oldtemp_deu['annualData']]
df_oldtemp_deu.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_oldtempdeu = df_oldtemp_deu.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_oldtempdeu

,Interval_Years,Annual_Data
0,1980 1999,7.67791


In [37]:
# precipitaciones en Alemania entre los años 1980 y 1999

In [38]:
deu_prec_1980_1999 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/pr/1980/1999/DEU')
df_oldprec_deu = json_normalize(deu_prec_1980_1999.json())
df_oldprec_deu['Interval_Years'] = '1980 1999'
df_oldprec_deu['Annual_Data'] = [i[0] for i in df_oldprec_deu['annualData']]
df_oldprec_deu.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_oldprecdeu = df_oldprec_deu.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_oldprecdeu

,Interval_Years,Annual_Data
0,1980 1999,929.598637


In [39]:
# temperatura en China entre los años 1980 y 1999

In [40]:
chn_temp_1980_1999 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/tas/1980/1999/CHN')
df_oldtemp_chn = json_normalize(chn_temp_1980_1999.json())
df_oldtemp_chn['Interval_Years'] = '1980 1999'
df_oldtemp_chn['Annual_Data'] = [i[0] for i in df_oldtemp_chn['annualData']]
df_oldtemp_chn.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_oldtempchn = df_oldtemp_chn.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_oldtempchn

,Interval_Years,Annual_Data
0,1980 1999,4.626097


In [41]:
# precipitaciones en China entre los años 1980 y 1999

In [42]:
chn_prec_1980_1999 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/pr/1980/1999/CHN')
df_oldprec_chn = json_normalize(chn_prec_1980_1999.json())
df_oldprec_chn['Interval_Years'] = '1980 1999'
df_oldprec_chn['Annual_Data'] = [i[0] for i in df_oldprec_chn['annualData']]
df_oldprec_chn.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_oldprecchn = df_oldprec_chn.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_oldprecchn

,Interval_Years,Annual_Data
0,1980 1999,808.955153


In [43]:
# temperatura en Reino Unido entre los años 1980 y 1999

In [44]:
gbr_temp_1980_1999 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/tas/1980/1999/GBR')
df_oldtemp_gbr = json_normalize(gbr_temp_1980_1999.json())
df_oldtemp_gbr['Interval_Years'] = '1980 1999'
df_oldtemp_gbr['Annual_Data'] = [i[0] for i in df_oldtemp_gbr['annualData']]
df_oldtemp_gbr.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_oldtempgbr = df_oldtemp_gbr.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_oldtempgbr

,Interval_Years,Annual_Data
0,1980 1999,9.121408


In [45]:
# precipitaciones en Reino Unido entre los años 1980 y 1999

In [46]:
gbr_prec_1980_1999 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/pr/1980/1999/GBR')
df_oldprec_gbr = json_normalize(gbr_prec_1980_1999.json())
df_oldprec_gbr['Interval_Years'] = '1980 1999'
df_oldprec_gbr['Annual_Data'] = [i[0] for i in df_oldprec_gbr['annualData']]
df_oldprec_gbr.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_oldprecgbr = df_oldprec_gbr.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_oldprecgbr

,Interval_Years,Annual_Data
0,1980 1999,988.845497


In [47]:
# temperatura en India entre los años 1980 y 1999

In [48]:
ind_temp_1980_1999 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/tas/1980/1999/IND')
df_oldtemp_ind = json_normalize(ind_temp_1980_1999.json())
df_oldtemp_ind['Interval_Years'] = '1980 1999'
df_oldtemp_ind['Annual_Data'] = [i[0] for i in df_oldtemp_ind['annualData']]
df_oldtemp_ind.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_oldtempind = df_oldtemp_ind.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_oldtempind

,Interval_Years,Annual_Data
0,1980 1999,21.990156


In [49]:
# precipitaciones en India entre los años 1980 y 1999

In [50]:
ind_prec_1980_1999 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/pr/1980/1999/IND')
df_oldprec_ind = json_normalize(ind_prec_1980_1999.json())
df_oldprec_ind['Interval_Years'] = '1980 1999'
df_oldprec_ind['Annual_Data'] = [i[0] for i in df_oldprec_ind['annualData']]
df_oldprec_ind.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_oldprecind = df_oldprec_ind.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_oldprecind

,Interval_Years,Annual_Data
0,1980 1999,1018.582287


In [51]:
# temperatura en Rusia entre los años 1980 y 1999

In [52]:
rus_temp_1980_1999 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/tas/1980/1999/RUS')
df_oldtemp_rus = json_normalize(rus_temp_1980_1999.json())
df_oldtemp_rus['Interval_Years'] = '1980 1999'
df_oldtemp_rus['Annual_Data'] = [i[0] for i in df_oldtemp_rus['annualData']]
df_oldtemp_rus.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_oldtemprus = df_oldtemp_rus.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_oldtemprus

,Interval_Years,Annual_Data
0,1980 1999,-7.352788


In [53]:
# precipitaciones en Rusia entre los años 1980 y 1999

In [54]:
rus_prec_1980_1999 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/pr/1980/1999/RUS')
df_oldprec_rus = json_normalize(rus_prec_1980_1999.json())
df_oldprec_rus['Interval_Years'] = '1980 1999'
df_oldprec_rus['Annual_Data'] = [i[0] for i in df_oldprec_rus['annualData']]
df_oldprec_rus.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_oldprecrus = df_oldprec_rus.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_oldprecrus

,Interval_Years,Annual_Data
0,1980 1999,497.024078


In [55]:
#Vamos a concatenar las temperaturas de estos paises

In [56]:
oldtemp_concat = pd.concat([grouped_oldtempusa, grouped_oldtempdeu, grouped_oldtempchn, grouped_oldtempgbr, grouped_oldtempind, grouped_oldtemprus  ])

In [57]:
oldtemp_concat

,Interval_Years,Annual_Data
0,1980 1999,6.344341
0,1980 1999,7.677910
0,1980 1999,4.626097
0,1980 1999,9.121408
0,1980 1999,21.990156
0,1980 1999,-7.352788


In [58]:
# el orden de los paises es el siguiente USA, Alemania, China, UK, India y Rusia
# ahora vamos a poner esos nombres en lugar de los años y resetear el index

In [59]:
oldtemp_concat.reset_index(inplace=True)
oldtemp_concat.drop(['index'], axis=1, inplace=True)
oldtemp_concat

,Interval_Years,Annual_Data
0,1980 1999,6.344341
1,1980 1999,7.677910
2,1980 1999,4.626097
3,1980 1999,9.121408
4,1980 1999,21.990156
5,1980 1999,-7.352788


In [60]:
oldtemp_concat['Country'] = 'USA', 'Germany', 'China', 'UK', 'India', 'Rusia'
oldtemp_concat.drop(['Interval_Years'], axis = 1, inplace=True)
oldtemp_concat

,Annual_Data,Country
0,6.344341,USA
1,7.677910,Germany
2,4.626097,China
3,9.121408,UK
4,21.990156,India
5,-7.352788,Rusia


In [61]:
# lo mismo con las precipitaciones

In [62]:
oldprec_concat = pd.concat([grouped_oldprecusa, grouped_oldprecdeu, grouped_oldprecchn, grouped_oldprecgbr, grouped_oldprecind, grouped_oldprecrus  ])
oldprec_concat.reset_index(inplace=True)
oldprec_concat.drop(['index'], axis=1, inplace=True)
oldprec_concat['Country'] = 'USA', 'Germany', 'China', 'UK', 'India', 'Rusia'
oldprec_concat.drop(['Interval_Years'], axis = 1, inplace=True)
oldprec_concat

,Annual_Data,Country
0,841.912621,USA
1,929.598637,Germany
2,808.955153,China
3,988.845497,UK
4,1018.582287,India
5,497.024078,Rusia


### Ahora vamos a ver predicciones de la temperatus a futuro, de estos mismo paises

In [63]:
# emepezamos cogiendo la prediccion de la temperatura de USA 

In [64]:
usa_temp_2020_2039 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/tas/2020/2039/USA')
df_newtemp_usa = json_normalize(usa_temp_2020_2039.json())
df_newtemp_usa['Interval_Years'] = '2020 2039'
df_newtemp_usa['Annual_Data'] = [i[0] for i in df_newtemp_usa['annualData']]
df_newtemp_usa.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_newtempusa = df_newtemp_usa.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_newtempusa

,Interval_Years,Annual_Data
0,2020 2039,7.483289


In [65]:
deu_temp_2020_2039 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/tas/2020/2039/DEU')
df_newtemp_deu = json_normalize(deu_temp_2020_2039.json())
df_newtemp_deu['Interval_Years'] = '2020 2039'
df_newtemp_deu['Annual_Data'] = [i[0] for i in df_newtemp_deu['annualData']]
df_newtemp_deu.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_newtempdeu = df_newtemp_deu.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_newtempdeu

,Interval_Years,Annual_Data
0,2020 2039,8.535085


In [66]:
chn_temp_2020_2039 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/tas/2020/2039/CHN')
df_newtemp_chn = json_normalize(chn_temp_2020_2039.json())
df_newtemp_chn['Interval_Years'] = '2020 2039'
df_newtemp_chn['Annual_Data'] = [i[0] for i in df_newtemp_chn['annualData']]
df_newtemp_chn.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_newtempchn = df_newtemp_chn.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_newtempchn

,Interval_Years,Annual_Data
0,2020 2039,5.728654


In [67]:
gbr_temp_2020_2039 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/tas/2020/2039/GBR')
df_newtemp_gbr = json_normalize(gbr_temp_2020_2039.json())
df_newtemp_gbr['Interval_Years'] = '2020 2039'
df_newtemp_gbr['Annual_Data'] = [i[0] for i in df_newtemp_gbr['annualData']]
df_newtemp_gbr.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_newtempgbr = df_newtemp_gbr.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_newtempgbr

,Interval_Years,Annual_Data
0,2020 2039,9.668244


In [68]:
ind_temp_2020_2039 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/tas/2020/2039/IND')
df_newtemp_ind = json_normalize(ind_temp_2020_2039.json())
df_newtemp_ind['Interval_Years'] = '2020 2039'
df_newtemp_ind['Annual_Data'] = [i[0] for i in df_newtemp_ind['annualData']]
df_newtemp_ind.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_newtempind = df_newtemp_ind.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_newtempind

,Interval_Years,Annual_Data
0,2020 2039,22.846838


In [69]:
rus_temp_2020_2039 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/tas/2020/2039/RUS')
df_newtemp_rus = json_normalize(rus_temp_2020_2039.json())
df_newtemp_rus['Interval_Years'] = '2020 2039'
df_newtemp_rus['Annual_Data'] = [i[0] for i in df_newtemp_rus['annualData']]
df_newtemp_rus.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_newtemprus = df_newtemp_rus.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_newtemprus

,Interval_Years,Annual_Data
0,2020 2039,-5.949377


In [70]:
# vamos a hacaer la misma operacion que hemos hecho con al registro de temperaturas y de precipitaciones
# vamos a concatenar todos estos registros y dejarlos de una manera mas presentable y lista para graficar

In [71]:
newtemp_concat = pd.concat([grouped_newtempusa, grouped_newtempdeu, grouped_newtempchn, grouped_newtempgbr, grouped_newtempind, grouped_newtemprus  ])
newtemp_concat.reset_index(inplace=True)
newtemp_concat.drop(['index'], axis=1, inplace=True)
newtemp_concat['Country'] = 'USA', 'Germany', 'China', 'UK', 'India', 'Rusia'
newtemp_concat.drop(['Interval_Years'], axis = 1, inplace=True)
newtemp_concat

,Annual_Data,Country
0,7.483289,USA
1,8.535085,Germany
2,5.728654,China
3,9.668244,UK
4,22.846838,India
5,-5.949377,Rusia


In [72]:
#vamos a hacer el de prediccion de precipitaciones

In [73]:
usa_prec_2020_2039 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/pr/2020/2039/USA')
df_newprec_usa = json_normalize(usa_prec_2020_2039.json())
df_newprec_usa['Interval_Years'] = '2020 2039'
df_newprec_usa['Annual_Data'] = [i[0] for i in df_newprec_usa['annualData']]
df_newprec_usa.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_newprecusa = df_newprec_usa.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_newprecusa

,Interval_Years,Annual_Data
0,2020 2039,859.000935


In [74]:
deu_prec_2020_2039 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/pr/2020/2039/DEU')
df_newprec_deu = json_normalize(deu_prec_2020_2039.json())
df_newprec_deu['Interval_Years'] = '2020 2039'
df_newprec_deu['Annual_Data'] = [i[0] for i in df_newprec_deu['annualData']]
df_newprec_deu.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_newprecdeu = df_newprec_deu.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_newprecdeu

,Interval_Years,Annual_Data
0,2020 2039,949.68479


In [75]:
chn_prec_2020_2039 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/pr/2020/2039/CHN')
df_newprec_chn = json_normalize(chn_prec_2020_2039.json())
df_newprec_chn['Interval_Years'] = '2020 2039'
df_newprec_chn['Annual_Data'] = [i[0] for i in df_newprec_chn['annualData']]
df_newprec_chn.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_newprecchn = df_newprec_chn.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_newprecchn

,Interval_Years,Annual_Data
0,2020 2039,817.829902


In [76]:
gbr_prec_2020_2039 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/pr/2020/2039/GBR')
df_newprec_gbr = json_normalize(gbr_prec_2020_2039.json())
df_newprec_gbr['Interval_Years'] = '2020 2039'
df_newprec_gbr['Annual_Data'] = [i[0] for i in df_newprec_gbr['annualData']]
df_newprec_gbr.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_newprecgbr = df_newprec_gbr.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_newprecgbr

,Interval_Years,Annual_Data
0,2020 2039,1005.959857


In [77]:
ind_prec_2020_2039 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/pr/2020/2039/IND')
df_newprec_ind = json_normalize(ind_prec_2020_2039.json())
df_newprec_ind['Interval_Years'] = '2020 2039'
df_newprec_ind['Annual_Data'] = [i[0] for i in df_newprec_ind['annualData']]
df_newprec_ind.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_newprecind = df_newprec_ind.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_newprecind

,Interval_Years,Annual_Data
0,2020 2039,1018.861217


In [78]:
rus_prec_2020_2039 = requests.get('http://climatedataapi.worldbank.org/climateweb/rest/v1/country/annualavg/pr/2020/2039/RUS')
df_newprec_rus = json_normalize(rus_prec_2020_2039.json())
df_newprec_rus['Interval_Years'] = '2020 2039'
df_newprec_rus['Annual_Data'] = [i[0] for i in df_newprec_rus['annualData']]
df_newprec_rus.drop(['fromYear', 'toYear', 'annualData'], axis=1, inplace=True)
grouped_newprecrus = df_newprec_rus.groupby(['Interval_Years'],as_index=False).agg({'Annual_Data':'mean'})
grouped_newprecrus

,Interval_Years,Annual_Data
0,2020 2039,520.475287


In [79]:
#vamos a concatenar

In [80]:
newprec_concat = pd.concat([grouped_newprecusa, grouped_newprecdeu, grouped_newprecchn, grouped_newprecgbr, grouped_newprecind, grouped_newprecrus  ])
newprec_concat.reset_index(inplace=True)
newprec_concat.drop(['index'], axis=1, inplace=True)
newprec_concat['Country'] = 'USA', 'Germany', 'China', 'UK', 'India', 'Rusia'
newprec_concat.drop(['Interval_Years'], axis = 1, inplace=True)
newprec_concat

,Annual_Data,Country
0,859.000935,USA
1,949.684790,Germany
2,817.829902,China
3,1005.959857,UK
4,1018.861217,India
5,520.475287,Rusia


### Vamos a coger los data frames que necesito y los voy a exportar como csv para cargarlos para la visualizcaión
#### Nos interesan los siguientes dataframes:

- df (el dataset original ya limpio)

- oldtemp_concat (el concatenado de las temperaturas de los años 80 hasta el 99)

- oldprec_concat (el concatenado de las precipitaciones de los años 80 hasta el 99)

- newtemp_concat (el concatenado de la predicción de las temperaturas desde el 2020 hasta el 2039)

- newprec_concat (el concatenado de la predicción de las precipitaciones desde el 2020 hasta el 2039)

In [81]:
df.to_csv('ghg-emissions_clean.csv')
oldtemp_concat.to_csv('old_temp.csv')
oldprec_concat.to_csv('old_prec.csv')
newtemp_concat.to_csv('prediction_temp.csv')
newprec_concat.to_csv('prediction_prec.csv')